In [1]:
from read_mem import *
import matplotlib.pyplot as plt
import pathlib

In [3]:
# filenames = ["/home/bastian/D1/registration/mrislurm/436844_log_python_srun.txt"]

path = pathlib.Path("/home/bastian/D1/registration/cubeslurm/")

# slurmids = [437139, 437140]

# slurmids += [437147, 437148]

slurmids = [437150]

slurmids += [437152, 437153]

slurmids += [437156, 437157]


path = pathlib.Path("/home/bastian/D1/registration/cubeslurm/")
slurmids = range(437160, 437164)

filenames = [path / (str(slurmid) + "_log_python_srun.txt") for slurmid in slurmids]

# filenames = ["/home/bastian/D1/registration/mrislurm/436856_log_python_srun.txt",
# "/home/bastian/D1/registration/mrislurm/436844_log_python_srun.txt"
# ]

minm, maxm = 1e16, 0
for filename in filenames:

    

    mema, outfoldername = read_memory(filename)

    outfoldername = outfoldername.replace("\n", "")
    outfoldername = outfoldername.replace(" ", "")
    print(outfoldername)
    print(mema[:, 0])

    # minm = min([np.min(mema[:, 1]), minm])

    minm = None

    plt.plot(mema[:,0], mema[:,1], label=outfoldername); plt.xlabel("lbfgs iteration"); plt.ylabel("Memory (TB)")

plt.legend()
plt.ylim(minm, None)
plt.show()

E10A0.01LBFGS10C4Projector


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed